### 1-dimensionale Datenflüsse

Einzelne Funktionsheiten werden in Modellen zu Datenflüssen mehrere Funktionseinheiten zusammengesetzt. Flow-Design nennt das auch "zusammenstecken" oder "verdrahten" in Anlehnung an elektronische Schaltungen, die übrigens ganz natürlich dem PoMO folgen. Aus 0-dimensionalen Modellen werden so 1-dimensionale: viele punktuelle Funktionseinheiten zusammen ergeben einen linearen Fluss.

![](images/df14.png)

Wenn der Output einer Funktionseinheit in der Form dem Input einer anderen entspricht, können beide in eine producer-consumer Beziehung gebracht werden. Sie spannen dann einen Transformationsfluss auf.

Die Übersetzung eines 1-dimensionalen Flusses ist trivial: die Funktionen der einzelnen Funktionseinheiten...

In [45]:
// Funktionseinheiten
int[] Map(string roman)
    => roman.ToCharArray()
            .Select(romanDigit => romanDigit switch {
                        'I' => 1, 'V' => 5, 'X' => 10,
                        'L' => 50, 'C' => 100, 'D' => 500,
                        'M' => 1000
                    })
            .ToArray();

int[] Negate(int[] values) {
    var negatedValues = (int[])values.Clone();
    for(var i=0; i<negatedValues.Length-1; i++)
    if (negatedValues[i]<negatedValues[i+1])
        negatedValues[i] *= -1;
    return negatedValues;
}

int Sum(IEnumerable<int> values) {
    var sum = 0;
    foreach(var v in values) sum += v;
    return sum;
}

 ...werden schlicht nacheinander aufgerufen. Der Compiler gibt Feedback, ob die Bausteine eines Datenflusses zueinander passen.

In [46]:
// Datenfluss
var values = Map("XIV");
values = Negate(values);
var decimalNumber = Sum(values);

display($"XIV={decimalNumber}");

XIV=14

In dieser Übersetzung verliert das Modell des Datenflusses zwar seine Eigenschaft der grundsätzlichen Asynchronizität seiner Transformationsschritte, doch das ist in den meisten Fällen unerheblich. Datenflüsse sind auch Datenflüsse, wenn sie synchron operieren. Das Modell hat trotzdem einen hohen Wert durch seine Abstraktion.

Die Funktionseinheiten, die im 1-dimensionalen Datenfluss verdrahtet werden, sind (zunächst) Operationen. Der Datenfluss stellt ihre Integration dar: alle einzelnen Transformationen bilden in spezifischer Weise zusammengesteckt eine größere Transformation.

#### Datenströme

Solche Übersetzung von 1-dimensionalen Flüssen als Lösungen von größeren Problemen in Form einer Abfolge von Lösungen kleinerer Probleme funktionieren auch für Ströme von Einzeldaten statt Collections.

Datenströme sind nützlich, wenn...

* ...die Zahl der zu erzeugenden Output-Datenelemente unbekannt und wahrscheinlich groß ist. In dem Fall soll vielleicht vermieden werden, alle Datenelemente in einer Collection im Hauptspeicher zu sammeln.
* ...die Generierung aller Output-Datenelemente erhebliche Zeit in Anspruch nimmt. In dem Fall sollen schon generierte Output-Daten schon downstream in Verarbeitung gehen, während weitere beschafft werden.
* ...ein Datenfluss besser zu verstehen und/oder zu übersetzen ist als Strom von Einzeldaten.

Als Beispiel dafür mag die Traversierung eines Verzeichnisbaumes dienen, in dem Dateien eines bestimmten Typs gesucht werden, um sie zu analysieren. Zu ermitteln ist die Gesamtanzahl der Worte in relevanten Dateien.

![](images/df15.png)

Die Zahl der zu prüfenden Dateien ist in so einem Szenario womöglich schwer abzuschätzen und/oder die Lösung ist leichter "zu denken", wenn die Analyse sich auf einzelne Dateien konzentriert.

Zuerst die Übersetzung der Funktionseinheiten, der Bausteine des Flusses. Jede ist nur eine kleine Operation mit wenigen Zeilen Code:

In [47]:
void EnumerateFiles(string path, Action<string> onFilename) {
    // Die Logik hier ist aufwändiger als nötig aus didaktischen Gründen, um einen Output-Strom zu erzeugen.
    var filenames = System.IO.Directory.GetFiles(path);
    foreach(var f in filenames) {
        onFilename(f);
    }
    
    var subdirectories = System.IO.Directory.GetDirectories(path);
    foreach(var d in subdirectories)
        EnumerateFiles(d,
              onFilename);
}

void FilterByEndOfFilename(string filename, string pattern, Action<string> onFilename) {
    if (filename.EndsWith(pattern))
        onFilename(filename);
}

string Load(string filename) 
    => System.IO.File.ReadAllText(filename);

string[] SplitIntoWords(string text)
    => text.Split(new[]{' ', '\n', '\t'}, StringSplitOptions.RemoveEmptyEntries);

void Print(string filename, int numberOfWords) {
    display($"{filename} with {numberOfWords} words");
}

Und dann die Übersetzung des Flusses. Die mag ein wenig gewöhnungsbedürftig sein in der Schachtelung der Lambda-Funktionen als continuations. Doch das verliert sich, wenn man es ein paar Mal geschrieben und gelesen hat. Der Gewinn der Gewöhnung liegt dann darin, eine geradlinige Übersetzung für Datenflüsse mit Strömen zu haben, die in jedem Schritt gut testbar ist.

In [49]:
EnumerateFiles("..",
    filename => FilterByEndOfFilename(filename, ".ipynb",
          filename => {
              var text = Load(filename);
              var words = SplitIntoWords(text);
              Print(filename, words.Length);
          })
);

../anforderungen.ipynb with 4475 words

../prozess.ipynb with 1827 words

../anforderung-logik-lücke.ipynb with 3276 words

../index.ipynb with 125 words

../entwurf/entwurf_1.ipynb with 3485 words

...

../.ipynb_checkpoints/anatomie_3-checkpoint.ipynb with 3372 words

../.ipynb_checkpoints/prozess-checkpoint.ipynb with 1827 words

../.ipynb_checkpoints/index-checkpoint.ipynb with 115 words

../.ipynb_checkpoints/anatomie_2-checkpoint.ipynb with 2549 words

../.ipynb_checkpoints/anforderungen-checkpoint.ipynb with 4473 words

`EnumerateFiles` erzeugt für eine Input-Nachricht einen Strom von Output-Nachrichten. `FilterByEndOfFilename` hingegen hat die Aufgabe, Input zu verschlucken, falls er nicht dem Filterkriterium entspricht. Das klingt sehr unterschiedlich, ist letztlich jedoch dasselbe: Der Strom dient in beiden Fällen zur Ausgabe einer unbekannten Zahl von Elementen, das können 1 oder 1000 sein - oder auch gar keines.

Eine Funktion, die ein Resultat mittels `return` liefert, ist gezwungen, eines zu produzieren, selbst wenn das `null` sein sollte. Eine Funktion, die ihr Resultat stattdessen über eine continuation "hinausschiebt", kann auch entscheiden, gar kein Resultat zu liefern! Deshalb ist die Übersetzung von Datenfluss-Funktionseinheiten in Funktionen mit continuation die universellere, wenn auch etwas aufwändigere.